### kineticsデータセットの動画をmp4でダウンロード

In [ ]:
import json
from pytube import YouTube
import shutil
import os
import pickle

useLabel = [
    'baby waking up',
    'carrying baby',
    'crawling baby'
    ]

def loadJson(jsonFile):
    targetDic = {}
    with open(jsonFile, 'r') as f:
        jsonDic = json.load(f)
        for key in jsonDic.keys():
            dic = jsonDic[key]
            label = dic['annotations']['label']
            if label not in useLabel:
                continue
            segment = dic['annotations']['segment']
            url = dic['url']
            if not label in targetDic.keys():
                targetDic[label] = {}
            targetDic[label][key] = [segment, url]
    return targetDic
            
def downloadVideos(dic, folder, label, maxNum):
    saveFolder = folder + os.sep + label
    if not os.path.exists(saveFolder):
        os.mkdir(saveFolder)
    
    num = 0
    for key in dic.keys():
        try:
            segment = dic[key][0]
            url = dic[key][1]

            # youtubeからmp4形式でダウンロード
            yt = YouTube(url)
            yt.streams.filter(file_extension='mp4').first().download()

            # ファイル名の記号がダウンロードファイルからは除かれている
            shutil.move(
                yt.title.replace("'", "").replace(".", "").replace("/", "").replace(",", "") \
                .replace('"', '').replace("#", "").replace("?", "").replace(":", "").replace("|", "") \
                .replace(";", "").replace("\"", "") + '.mp4', 
                'input.mp4')                

            # ダウンロードした動画を指定秒数で切り抜き
            sec = int(segment[0])
            if sec < 60:         
                start = '00:00:' + str(int(segment[0])) + '.0'
            else:
                minutes = sec // 60
                sec2 = sec - minutes*60
                start = '00:' + str(minutes) + ':' + str(sec2)

            os.system('ffmpeg -y -i input.mp4'  + ' -ss ' + start + ' -t 00:00:10.0' + ' out.mp4')

            # 作成した動画をラベル名のフォルダに移動
            moviePath = saveFolder + os.sep + key + '.mp4'
            shutil.move('out.mp4', moviePath) 
            num += 1
                
            if num >= maxNum:
                return
            
            if num % 10 == 0:
                print(str(num) + 'ファイル完了')
        except Exception as e:
            print(key + ":" + str(e))

#### trainのダウンロード

In [ ]:
targetDic = loadJson('kinetics_train.json')

for label in useLabel:
    dic = targetDic[label]
    with open(label + 'Datas.p', mode='wb') as f:
        pickle.dump(dic, f)

In [ ]:
for label in useLabel:
    with open(label + 'Datas.p', mode='rb') as f:
        dic = pickle.load(f)
    downloadVideos(dic, 'kineticsMovies', label, 500)

#### valのダウンロード

In [ ]:
targetValDic = loadJson('kinetics_val.json')
for label in useLabel:
    dic = targetValDic[label]
    with open(label + 'valDatas.p', mode='wb') as f:
        pickle.dump(dic, f)

In [ ]:
for label in useLabel:
    with open(label + 'valDatas.p', mode='rb') as f:
        dic = pickle.load(f)
    downloadVideos(dic, 'kineticsMovies2', label, 500)